# Reading the ROOT files

In [1]:
exec(open("./do_imports.py").read())

def get_paired_data(data):
    next_ts = data.timestamp[1:]
    next_ts = ak.concatenate([next_ts, 0])

    condition_1 = next_ts - data.timestamp<600
    condition_2 = next_ts - data.timestamp>0
    condition_3 = data.dt_next_us < 600

    both = condition_1 * condition_2 * condition_3

    first_of_two = ak.where(both)[0]
    signal1 = data[first_of_two]
    signal2 = data[first_of_two+1]
    
    return signal1, signal2
ibd = ak.from_json('data/ibd.json')
fastn = ak.from_json('data/fastn.json')

fn0, fn1 = get_paired_data(fastn)
ibd0, ibd1 = get_paired_data(ibd)

In [410]:
selection = [ibd0, fn0]
samples = min([len(i) for i in selection])
first_signal = ak.concatenate([fn0[:samples], ibd0[:samples]])
second_signal = ak.concatenate([fn1[:samples], ibd1[:samples]])
y = np.where(first_signal.code==2, 0, 1)

In [411]:
trainidx, testidx = train_test_split(np.arange(len(y)), test_size=.25, random_state=43)

In [412]:
with open('./data/test_water_og.pkl', 'rb') as f:
    data = pickle.load(f)

In [413]:
train = dict(ev1 = first_signal[trainidx],
             ev2 = second_signal[trainidx],
             y = y[trainidx]
            )
test = dict(ev1 = first_signal[testidx],
             ev2 = second_signal[testidx],
             y = y[testidx]
            )

# pickle.dump(test, open( "./data/test_water_og.pkl", "wb" ))

In [62]:
fastn = uproot.open('data/fastn_water.root'+':data')
fastn= fastn.arrays(library='awkward')

# Time-sort all the data arrays for recurrent/sequential purposes. 
for j, data in enumerate([fastn]):
    print('set %i'%(j))
    args = ak.argsort(data['hittime'])
    for key in ['hittime', 'pmtcharge', 'channel']:
        data[key] = data[key][args]
        
# Get rid of the first and last events of each run 
fastn = fastn[fastn['inner_hit_prev']>0]
fastn = fastn[fastn['inner_hit_next']>0]
fastn = fastn[fastn['dt_prev_us']>0]
ak.to_json(fastn, 'data/fastn_water.json')

In [3]:
exec(open("./do_imports.py").read())

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

In [371]:
from evaluate_results import *

We want where quicknext<500 us (so another prompt follows) and the subid==1. 

What if we boost our fast-neutron stamps by not requiring it to be the first event (so tri-neutron events recorded too but in pairs?) 
<br>
Obviously this is not realistic since we'd veto these by having 3, but we can at least pretend they're di-neutrons! 

Edit: the next code is already doing that! We gain 1149/5500 of the stats from that trick, which is necessary. 

In [56]:
with open('./data/test_water_og.pkl', 'rb') as f:
    data = pickle.load(f)

### Prep data for model input

In [20]:
from batched_data import *

In [423]:
val = DatSequence('test_water_og', batch_size=10, shuffle=False)

In [424]:
X, y = val.__getitem__(0)

The error is between first_b and X_test1 at the 2nd index, not the 0 or 1. 

In [504]:
new_bins1 = ak.values_astype((first_signal.hittime-300)/div_factor, np.int32)
new_bins2 = ak.values_astype((second_signal.hittime-300)/div_factor, np.int32)
first = np.zeros((len(first_signal), npmts, nbins), dtype=np.float32)
second = np.zeros_like(first)

for i in range(0, len(first_signal)):
    first[i, first_signal.channel[i], new_bins1[i]]=first_signal.pmtcharge[i]
    second[i, second_signal.channel[i], new_bins2[i]]=second_signal.pmtcharge[i]
    
# X_train1, X_test1, X_train2, X_test2, y_train, y_test = train_test_split(
#     first, second, y,
#     test_size=0.25, random_state=43
# )

In [510]:
selection = [ibd0, fn0]
samples = min([len(i) for i in selection])

npmts = 2330
timetot = 1500 #ns 
tres = 10 # ns 
div_factor = tres 
nbins = int(timetot/tres)
size = samples * len(selection)
y = np.array([]) 

first = np.zeros((size, npmts, nbins), dtype=np.float32)
second = np.zeros_like(first)

data_to_manipulate = [[fn0[:samples], fn1[:samples]], [ibd0[:samples], ibd1[:samples]]]
for d, (signal1, signal2) in enumerate(data_to_manipulate):
    assert ak.all(signal1.code==signal2.code)
    print('set %i of %i'%(d+1, len(data_to_manipulate)))
    
    new_bins1 = ak.values_astype((signal1.hittime-300)/div_factor, np.int32)
    new_bins2 = ak.values_astype((signal2.hittime-300)/div_factor, np.int32)
    
    for i in range(0, samples):
        first[d*samples+i, signal1.channel[i], new_bins1[i]]=signal1.pmtcharge[i]
        second[d*samples+i, signal2.channel[i], new_bins2[i]]=signal2.pmtcharge[i]
    y=np.append(y, d*np.ones(samples))
    
    if d==0: bins_out = new_bins1
    else: bins_out = ak.concatenate([bins_out, new_bins1])
    
X_train1, X_test1, X_train2, X_test2, y_train, y_test = train_test_split(
    first, second, y,
    test_size=0.25, random_state=43
)

set 1 of 2
set 2 of 2
